# Knapsack Problem

In the knapsack problem, we are given the set of items $N = \{ 0, ..., n-1 \}$ with weights $w_i$ and profits $p_i$ for $i \in N$ and a knapsack with capacity $c$.
We want to maximize the total profit of the items in the knapsack.

## DP Formulation

Let $V(r, i)$ be the maximum profit of selecting items to pack from $\{ i, ..., n - 1 \}$ into a knapsack with capacity $r$.
The DP formulation is as follows:

$$
\begin{align}
    \text{compute } & V(c, 0) \\
    & V(r, i) = \begin{cases}
        \max\{ p_i + V(r - w_i, i + 1), V(r, i + 1) \} & \text{if } i < n \land r \geq w_i \\
        V(r, i + 1) & \text{if } i < n \land r < w_i \\
        0 & \text{otherwise.}
    \end{cases}
\end{align}
$$

## Data

In [1]:
n = 4
weights = [10, 20, 30, 40]
profits = [5, 25, 35, 50]
capacity = 50

## Modeling

In [2]:
# !pip install didppy

In [3]:
import didppy as dp


model = dp.Model(maximize=True, float_cost=False)

item = model.add_object_type(number=n)
r = model.add_int_var(target=capacity)
i = model.add_element_var(object_type=item, target=0)

w = model.add_int_table(weights)
p = model.add_int_table(profits)

pack = dp.Transition(
    name="pack",
    cost=p[i] + dp.IntExpr.state_cost(),
    effects=[(r, r - w[i]), (i, i + 1)],
    preconditions=[i < n, r >= w[i]],
)
model.add_transition(pack)

ignore = dp.Transition(
    name="ignore",
    cost=dp.IntExpr.state_cost(),
    effects=[(i, i + 1)],
    preconditions=[i < n],
)
model.add_transition(ignore)

model.add_base_case([i == n])

## Solving with ForwardRecursion

In [4]:
solver = dp.ForwardRecursion(model)
solution = solver.search()

for i, t in enumerate(solution.transitions):
    if t.name == "pack":
        print("pack {}".format(i))

print("profit: {}".format(solution.cost))

pack 1
pack 2
profit: 60


## Solving with CABS

In [5]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

for i, t in enumerate(solution.transitions):
    if t.name == "pack":
        print("pack {}".format(i))

print("")
print("Profit: {}".format(solution.cost))

pack 1
pack 2

Profit: 60
